In [1]:
import numpy as np
DTYPE = np.float64
import matplotlib.pyplot as plt
import pickle
import os
from tqdm import tqdm
import sys
sys.path.insert(0, "../../packages")
import gzip

In [2]:
phi_globals = [
              # np.array([0.1, 0.2, 0.3, 0.4], dtype=DTYPE),
              # np.array([0.1, 0.2, 0.4, 0.3], dtype=DTYPE),
              # np.array([0.1, 0.3, 0.2, 0.4], dtype=DTYPE),
              # np.array([0.1, 0.3, 0.4, 0.2], dtype=DTYPE),
              # np.array([0.1, 0.4, 0.2, 0.3], dtype=DTYPE),
              # np.array([0.1, 0.4, 0.3, 0.2], dtype=DTYPE),
              # np.array([0.2, 0.1, 0.3, 0.4], dtype=DTYPE),
              # np.array([0.2, 0.1, 0.4, 0.3], dtype=DTYPE),
              # np.array([0.2, 0.3, 0.1, 0.4], dtype=DTYPE),
              # np.array([0.2, 0.3, 0.4, 0.1], dtype=DTYPE),
              # np.array([0.2, 0.4, 0.3, 0.1], dtype=DTYPE),
              # np.array([0.2, 0.4, 0.1, 0.3], dtype=DTYPE),
              # np.array([0.3, 0.1, 0.2, 0.4], dtype=DTYPE),
              # np.array([0.3, 0.1, 0.4, 0.2], dtype=DTYPE),
              # np.array([0.3, 0.2, 0.1, 0.4], dtype=DTYPE),
              # np.array([0.3, 0.2, 0.4, 0.1], dtype=DTYPE),
              # np.array([0.3, 0.4, 0.1, 0.2], dtype=DTYPE),
              # np.array([0.3, 0.4, 0.2, 0.1], dtype=DTYPE),
              # np.array([0.4, 0.1, 0.2, 0.3], dtype=DTYPE),
              # np.array([0.4, 0.1, 0.3, 0.2], dtype=DTYPE),
              # np.array([0.4, 0.2, 0.1, 0.3], dtype=DTYPE),
              # np.array([0.4, 0.2, 0.3, 0.1], dtype=DTYPE),
              # np.array([0.4, 0.3, 0.1, 0.2], dtype=DTYPE),
              np.array([0.4, 0.3, 0.2, 0.1], dtype=DTYPE)
             ]

Xs = np.arange(1, 10.1, 0.1)

steps = 100000


In [3]:
# Flory Huggins Free Energy function
def floryHuggins(phi:DTYPE, chi:np.array):
    part_1 = np.sum(phi*np.log(phi))
    part_2 = 0

    for i in range(len(phi)):
        for j in range(i+1, len(phi)):
            part_2 += chi[i][j]*phi[i]*phi[j]

    return part_1 + part_2

In [4]:
# def are_locations_equal(loc1, loc2, tolerance=1e-2):
#     # Compare the numpy arrays [x, y, z]
#     arr_equal = np.allclose(loc1[0], loc2[0], atol=tolerance)
#     # Compare the floats [t]
#     t_equal = np.allclose(loc1[1], loc2[1], atol=tolerance)
#     return arr_equal and t_equal


def are_locations_equal(loc1, loc2, tolerance=1e-2):
    """
    Compare two locations where:
    - loc[0] is a 2D array (rows may be swapped).
    - loc[1] is a 1D array (elements may be swapped).
    """
    # Compare the 2D arrays (row order insensitive)
    arr1, arr2 = loc1[0], loc2[0]
    if arr1.shape != arr2.shape:
        return False
    # Check if all rows of arr1 exist in arr2 (within tolerance)
    row_matched = np.zeros(arr1.shape[0], dtype=bool)
    for row1 in arr1:
        for i, row2 in enumerate(arr2):
            if not row_matched[i] and np.allclose(row1, row2, atol=tolerance):
                row_matched[i] = True
                break
    arr_equal = np.all(row_matched)

    # Compare the 1D arrays (element order insensitive)
    vec1, vec2 = loc1[1], loc2[1]
    if vec1.shape != vec2.shape:
        return False
    # Check if all elements of vec1 exist in vec2 (within tolerance)
    vec_matched = np.zeros(vec1.shape[0], dtype=bool)
    for val1 in vec1:
        for i, val2 in enumerate(vec2):
            if not vec_matched[i] and np.isclose(val1, val2, atol=tolerance):
                vec_matched[i] = True
                break
    vec_equal = np.all(vec_matched)

    return arr_equal and vec_equal

In [5]:
def format_key(key):
    if isinstance(key[0], tuple):  # For nested tuples like ((1,2),(3,4))
        return '[' + ' '.join(str(x) for x in key[0]) + '][' + ' '.join(str(x) for x in key[1]) + ']'
    else:  # For flat tuples like (1,2,3)
        return '[' + ' '.join(str(x) for x in key) + ']'


In [6]:
for phi_global in phi_globals:
    F_4_phase = []
    for X in (Xs):
        # Load the distinct 4 phase solution!
        input_filepath = f"data/withFlory/phi_g{phi_global}/raw/X{X:.3f}/"
        input_filename = f"initial_system.pkl"
        input_file = os.path.join(input_filepath, input_filename)
        
        with gzip.open(input_file, 'rb') as f:
            loaded_data = pickle.load(f)

        f = 0
        for _ in range(len(loaded_data["flory_vols"])):
            f += loaded_data["flory_vols"][_]*floryHuggins(loaded_data["flory_phases"][_], loaded_data["chis"])
        F_4_phase.append(f)        

In [7]:
#######################################
# Store the initial F values for 4->2
#######################################
initial_F_4To2 = {}

#######################################
# Store the best F values  for 4->2
#######################################
best_F_4To2 = {}

#######################################
# Store the initial F values for 4->3
#######################################
initial_F_4To3 = {}

#######################################
# Store the initial F values for 4->3
#######################################
best_F_4To3 = {}




#######################################
# Global bounds for plotting purposes
#######################################
all_best_F_values = []

In [8]:
#######################################
# 4 to 2
#######################################

#######################
# 4->2, combine 2+2
#######################

merged_compartments_set1 = [np.array([1, 2], dtype=np.int64), 
                            np.array([1, 3], dtype=np.int64),
                            np.array([1, 4], dtype=np.int64)]

merged_compartments_set2 = [np.array([3, 4], dtype=np.int64), 
                            np.array([2, 4], dtype=np.int64),
                            np.array([2, 3], dtype=np.int64)]

for merged_compartments in (zip(merged_compartments_set1, merged_compartments_set2)):
    key = (tuple(merged_compartments[0]), tuple(merged_compartments[1]))
    f_in = []
    f_best = []
    
    for phi_global in phi_globals:
        for X in tqdm(Xs):
            input_filepath = f"data/withFlory/phi_g{phi_global}/raw/X{X:.3f}/steps{steps}/mergers/4To2_combine2And2/{merged_compartments[0]} {merged_compartments[1]}/"
            input_filename = f"initial_and_best.pkl"
            input_file = os.path.join(input_filepath, input_filename)
            
            with gzip.open(input_file, "rb") as file:
                loaded_data = pickle.load(file)

            f_in.append(loaded_data["F_initial"])
            f_best.append(loaded_data["F_best"])

    #######################
    #  Update the dictionaries
    #######################
    initial_F_4To2[key] = f_in
    best_F_4To2[key] = f_best           

    all_best_F_values.extend(f_best)
    
#######################
# 4->2, combine 3
#######################

merged_compartments_list = [np.array([1, 2, 3], dtype=np.int64), 
                            np.array([1, 2, 4], dtype=np.int64),
                            np.array([1, 3, 4], dtype=np.int64),
                            np.array([2, 3, 4], dtype=np.int64)]

for merged_compartments in merged_compartments_list:
    key = tuple(merged_compartments)
    f_in = []
    f_best = []
    
    for phi_global in phi_globals:
        for X in tqdm(Xs):

            input_filepath = f"data/withFlory/phi_g{phi_global}/raw/X{X:.3f}/steps{steps}/mergers/4To2_combine3/{merged_compartments}/"
            input_filename = f"initial_and_best.pkl"
            input_file = os.path.join(input_filepath, input_filename)
            
            with gzip.open(input_file, "rb") as file:
                loaded_data = pickle.load(file)

            
            f_in.append(loaded_data["F_initial"])
            f_best.append(loaded_data["F_best"])

    initial_F_4To2[key] = f_in
    best_F_4To2[key] = f_best     
    all_best_F_values.extend(f_best)
    

100%|███████████████████████████████████████████| 91/91 [00:00<00:00, 8709.22it/s]


In [9]:
#######################################
# 4 to 3
#######################################

#######################
# 4->3, combine 2
#######################
merged_compartments_list = [np.array([1, 2], dtype=np.int64),
                            np.array([1, 3], dtype=np.int64),
                            np.array([1, 4], dtype=np.int64),
                            np.array([2, 3], dtype=np.int64),
                            np.array([2, 4], dtype=np.int64),
                            np.array([3, 4], dtype=np.int64),
                           ]

for merged_compartments in merged_compartments_list:
    key = tuple(merged_compartments)
    
    f_in = []
    f_best = []
    
    for phi_global in phi_globals:
        for X in tqdm(Xs):

            input_filepath = f"data/withFlory/phi_g{phi_global}/raw/X{X:.3f}/steps{steps}/mergers/4To3_combine2/{merged_compartments}/"
            input_filename = f"initial_and_best.pkl"
            input_file = os.path.join(input_filepath, input_filename)
            
            with gzip.open(input_file, "rb") as file:
                loaded_data = pickle.load(file)

            f_in.append(loaded_data["F_initial"])

            f_best.append(loaded_data["F_best"])

    initial_F_4To3[key] = f_in
    best_F_4To3[key] = f_best     
    all_best_F_values.extend(f_best)
    

100%|███████████████████████████████████████████| 91/91 [00:00<00:00, 9072.32it/s]


In [10]:
global_best_ymin = np.min(all_best_F_values)
global_best_ymax = np.max(all_best_F_values)

padding = 0.05 * (global_best_ymax - global_best_ymin)
global_best_ymin -= padding
global_best_ymax += padding

In [11]:
##########################
# 4 to 2
##########################



colors = plt.cm.tab10.colors  # Uses matplotlib's default color cycle (10 colors)
fig, ax = plt.subplots(figsize=(4, 3))

ax.scatter(Xs, F_4_phase, color = "k", marker = "x", label = "4-phase", s=5)
for ctr, (key, vals) in enumerate(best_F_4To2.items()):
    legend_label = format_key(key)
    ax.scatter(Xs, vals, label=legend_label, color=colors[ctr % len(colors)], marker = ".", s = 8)
    
legend = ax.legend(loc='lower right', fontsize=8, frameon=True)
for text in legend.get_texts():
    text.set_fontweight("bold") 

ax.set_ylabel(r"$\mathbf{F \,(k_BT)}$", fontsize=12, fontweight="bold")
ax.set_xlabel(r"$\mathbf{X}$", fontsize=12, fontweight="bold")

ax.set_xticks(Xs[::10])

for label in ax.get_xticklabels() + ax.get_yticklabels():
    label.set_fontweight("bold")
                        
ax.tick_params(axis='both', which='both', direction='in', width=1.5, length=3.5, labelsize=10)
ax.tick_params(axis='both', which='minor', direction='in', width=0.5, length=2, labelsize=8)

ax.set_ylim(global_best_ymin, global_best_ymax)
ax.grid(True, which='major', linestyle='--', linewidth=1, alpha=0.6)

plt.tight_layout()

output_filepath = f"data/withFlory/phi_g{phi_global}/analysed/steps{steps}/"
output_filename = f"F_4To2.png"
if not os.path.exists(output_filepath):
    os.makedirs(output_filepath)
output_file = os.path.join(output_filepath, output_filename)
plt.savefig(output_file,  dpi=400)
plt.close()

print(f"Saved @ {output_file}")

Saved @ data/withFlory/phi_g[0.4 0.3 0.2 0.1]/analysed/steps100000/F_4To2.png


In [12]:
##########################
# 4 to 3
##########################

colors = plt.cm.tab10.colors  # Uses matplotlib's default color cycle (10 colors)
fig, ax = plt.subplots(figsize=(4, 3))
ax.scatter(Xs, F_4_phase, color = "k", marker = "x", label = "4-phase", s=5)

for ctr, (key, vals) in enumerate(best_F_4To3.items()):
    legend_label = format_key(key)
    ax.scatter(Xs, vals, label=legend_label, color=colors[ctr % len(colors)], marker = ".", s=8)
    
legend = ax.legend(loc='lower right', fontsize=8, frameon=True)
for text in legend.get_texts():
    text.set_fontweight("bold") 

ax.set_ylabel(r"$\mathbf{F \,(k_BT)}$", fontsize=12, fontweight="bold")
ax.set_xlabel(r"$\mathbf{X}$", fontsize=12, fontweight="bold")

ax.set_xticks(Xs[::10])

for label in ax.get_xticklabels() + ax.get_yticklabels():
    label.set_fontweight("bold")
                        
ax.tick_params(axis='both', which='both', direction='in', width=1.5, length=3.5, labelsize=10)
ax.tick_params(axis='both', which='minor', direction='in', width=0.5, length=2, labelsize=8)

ax.set_ylim(global_best_ymin, global_best_ymax)
ax.grid(True, which='major', linestyle='--', linewidth=1, alpha=0.6)

plt.tight_layout()

output_filepath = f"data/withFlory/phi_g{phi_global}/analysed/steps{steps}/"
output_filename = f"F_4To3.png"
if not os.path.exists(output_filepath):
    os.makedirs(output_filepath)
output_file = os.path.join(output_filepath, output_filename)
plt.savefig(output_file,  dpi=400)
plt.close()

print(f"Saved @ {output_file}")

Saved @ data/withFlory/phi_g[0.4 0.3 0.2 0.1]/analysed/steps100000/F_4To3.png


In [13]:
#################################
# Distinct mergers for 4 to 3
#################################

for phi_global in phi_globals:
    count_distinct_mergers = []
    count_distinct_Fs = []
    for X in Xs:
        distinct_mergers = []
        grouped_best = []
        grouped_best_F = []

        merged_compartments_list = [np.array([1, 2], dtype=np.int64),
                            np.array([1, 3], dtype=np.int64),
                            np.array([1, 4], dtype=np.int64),
                            np.array([2, 3], dtype=np.int64),
                            np.array([2, 4], dtype=np.int64),
                            np.array([3, 4], dtype=np.int64),
                           ]

        for merged_compartments in merged_compartments_list:
            input_filepath = f"data/withFlory/phi_g{phi_global}/raw/X{X:.3f}/steps{steps}/mergers/4To3_combine2/{merged_compartments}/"
            input_filename = f"initial_and_best.pkl"
            input_file = os.path.join(input_filepath, input_filename)
            
            try:
                with gzip.open(input_file, "rb") as file:
                    loaded_data = pickle.load(file)
            
            except FileNotFoundError:
                print(f"File not found: {input_file}, skipping...")
                continue  # if this is inside a loop

            grouped_best.append(loaded_data["best_location"])
            # grouped_best_F.append(loaded_data["F_best"])
            grouped_best_F.append(round(loaded_data["F_best"], 4))
        count_distinct_Fs.append(len(set(grouped_best_F)))
            
        # print(grouped_best)
        # print()
        # print()

        for loc in grouped_best:
            is_unique=True
            for seen_loc in distinct_mergers:
                if are_locations_equal(loc, seen_loc):
                    is_unique = False
                    break
            if is_unique:
                distinct_mergers.append(loc)
        count_distinct_mergers.append(len(distinct_mergers))

In [14]:
#################################
# Distinct mergers for 4 to 3 Free Energy
#################################
fig, ax = plt.subplots(figsize=(4, 3))

ax.scatter(Xs, count_distinct_Fs, marker = "o", s = 5)

ax.set_ylabel(r"Distinct Mergers", fontsize=12, fontweight="bold")
ax.set_xlabel(r"$\mathbf{X}$", fontsize=12, fontweight="bold")

ax.set_xticks(Xs[::10])
ax.set_yticks(range(0, max(count_distinct_Fs) + 2))

for label in ax.get_xticklabels() + ax.get_yticklabels():
    label.set_fontweight("bold")
                        
ax.tick_params(axis='both', which='both', direction='in', width=1.5, length=3.5, labelsize=10)
ax.tick_params(axis='both', which='minor', direction='in', width=0.5, length=2, labelsize=8)

# ax.grid(True, which='major', linestyle='--', linewidth=1, alpha=0.6)

plt.tight_layout()

output_filepath = f"data/withFlory/phi_g{phi_global}/analysed/steps{steps}/"
output_filename = f"distinct_mergers_4To3_Free Energy.png"
if not os.path.exists(output_filepath):
    os.makedirs(output_filepath)
output_file = os.path.join(output_filepath, output_filename)
plt.savefig(output_file,  dpi=400)
plt.close()

print(f"Saved @ {output_file}")

Saved @ data/withFlory/phi_g[0.4 0.3 0.2 0.1]/analysed/steps100000/distinct_mergers_4To3_Free Energy.png


In [15]:
#################################
# Distinct mergers for 4 to 3 Positional
#################################

fig, ax = plt.subplots(figsize=(4, 3))

ax.scatter(Xs, count_distinct_mergers, marker = "o", s = 5)

ax.set_ylabel(r"Distinct Mergers", fontsize=12, fontweight="bold")
ax.set_xlabel(r"$\mathbf{X}$", fontsize=12, fontweight="bold")

ax.set_xticks(Xs[::10])
ax.set_yticks(range(0, max(count_distinct_mergers) + 2))

for label in ax.get_xticklabels() + ax.get_yticklabels():
    label.set_fontweight("bold")
                        
ax.tick_params(axis='both', which='both', direction='in', width=1.5, length=3.5, labelsize=10)
ax.tick_params(axis='both', which='minor', direction='in', width=0.5, length=2, labelsize=8)

# ax.grid(True, which='major', linestyle='--', linewidth=1, alpha=0.6)

plt.tight_layout()

output_filepath = f"data/withFlory/phi_g{phi_global}/analysed/steps{steps}/"
output_filename = f"distinct_mergers_4To3_Position.png"
if not os.path.exists(output_filepath):
    os.makedirs(output_filepath)
output_file = os.path.join(output_filepath, output_filename)
plt.savefig(output_file,  dpi=400)
plt.close()

print(f"Saved @ {output_file}")

Saved @ data/withFlory/phi_g[0.4 0.3 0.2 0.1]/analysed/steps100000/distinct_mergers_4To3_Position.png


In [16]:
#################################
# Distinct mergers for 4 to 2
#################################
for phi_global in phi_globals:
    count_distinct_mergers = []
    count_distinct_Fs = []
    for X in Xs:
        distinct_mergers = []
        grouped_best = []
        grouped_best_F = []
        
        merged_compartments_set1 = [np.array([1, 2], dtype=np.int64), 
                                    np.array([1, 3], dtype=np.int64),
                                    np.array([1, 4], dtype=np.int64)]
        
        merged_compartments_set2 = [np.array([3, 4], dtype=np.int64), 
                                    np.array([2, 4], dtype=np.int64),
                                    np.array([2, 3], dtype=np.int64)]
        
        for merged_compartments in (zip(merged_compartments_set1, merged_compartments_set2)):
            input_filepath = f"data/withFlory/phi_g{phi_global}/raw/X{X:.3f}/steps{steps}/mergers/4To2_combine2And2/{merged_compartments[0]} {merged_compartments[1]}/"
            input_filename = f"initial_and_best.pkl"
            input_file = os.path.join(input_filepath, input_filename)
            
            try:
                with gzip.open(input_file, "rb") as file:
                    loaded_data = pickle.load(file)
            
            except FileNotFoundError:
                print(f"File not found: {input_file}, skipping...")
                continue  # if this is inside a loop

            grouped_best.append(loaded_data["best_location"])
            # grouped_best_F.append(loaded_data["F_best"])
            grouped_best_F.append(round(loaded_data["F_best"], 4))
            

        merged_compartments_list = [np.array([1, 2, 3], dtype=np.int64), 
                                    np.array([1, 2, 4], dtype=np.int64),
                                    np.array([1, 3, 4], dtype=np.int64),
                                    np.array([2, 3, 4], dtype=np.int64)]

        for merged_compartments in merged_compartments_list:
            input_filepath = f"data/withFlory/phi_g{phi_global}/raw/X{X:.3f}/steps{steps}/mergers/4To2_combine3/{merged_compartments}/"
            input_filename = f"initial_and_best.pkl"
            input_file = os.path.join(input_filepath, input_filename)
            
            try:
                with gzip.open(input_file, "rb") as file:
                    loaded_data = pickle.load(file)
            
            except FileNotFoundError:
                print(f"File not found: {input_file}, skipping...")
                continue  # if this is inside a loop

            grouped_best.append(loaded_data["best_location"])
            grouped_best_F.append(round(loaded_data["F_best"], 4))

        count_distinct_Fs.append(len(set(grouped_best_F)))
        
        for loc in grouped_best:
            is_unique=True
            for seen_loc in distinct_mergers:
                if are_locations_equal(loc, seen_loc):
                    is_unique = False
                    break
            if is_unique:
                distinct_mergers.append(loc)
        count_distinct_mergers.append(len(distinct_mergers))

In [17]:
#################################
# Distinct mergers for 4 to 2 Free Energy
#################################
fig, ax = plt.subplots(figsize=(4, 3))

ax.scatter(Xs, count_distinct_Fs, marker = "o", s = 5)

ax.set_ylabel(r"Distinct Mergers", fontsize=12, fontweight="bold")
ax.set_xlabel(r"$\mathbf{X}$", fontsize=12, fontweight="bold")

ax.set_xticks(Xs[::10])
ax.set_yticks(range(0, max(count_distinct_Fs) + 2))

for label in ax.get_xticklabels() + ax.get_yticklabels():
    label.set_fontweight("bold")
                        
ax.tick_params(axis='both', which='both', direction='in', width=1.5, length=3.5, labelsize=10)
ax.tick_params(axis='both', which='minor', direction='in', width=0.5, length=2, labelsize=8)

# ax.grid(True, which='major', linestyle='--', linewidth=1, alpha=0.6)

plt.tight_layout()

output_filepath = f"data/withFlory/phi_g{phi_global}/analysed/steps{steps}/"
output_filename = f"distinct_mergers_4To2_Free_Energy.png"
if not os.path.exists(output_filepath):
    os.makedirs(output_filepath)
output_file = os.path.join(output_filepath, output_filename)
plt.savefig(output_file,  dpi=400)
plt.close()

print(f"Saved @ {output_file}")

Saved @ data/withFlory/phi_g[0.4 0.3 0.2 0.1]/analysed/steps100000/distinct_mergers_4To2_Free_Energy.png


In [18]:
#################################
# Distinct mergers for 4 to 2 Positional
#################################

fig, ax = plt.subplots(figsize=(4, 3))

ax.scatter(Xs, count_distinct_mergers, marker = "o", s = 5)

ax.set_ylabel(r"Distinct Mergers", fontsize=12, fontweight="bold")
ax.set_xlabel(r"$\mathbf{X}$", fontsize=12, fontweight="bold")

ax.set_xticks(Xs[::10])
ax.set_yticks(range(0, max(count_distinct_mergers) + 2))

for label in ax.get_xticklabels() + ax.get_yticklabels():
    label.set_fontweight("bold")
                        
ax.tick_params(axis='both', which='both', direction='in', width=1.5, length=3.5, labelsize=10)
ax.tick_params(axis='both', which='minor', direction='in', width=0.5, length=2, labelsize=8)

# ax.grid(True, which='major', linestyle='--', linewidth=1, alpha=0.6)

plt.tight_layout()

output_filepath = f"data/withFlory/phi_g{phi_global}/analysed/steps{steps}/"
output_filename = f"distinct_mergers_4To2_Position.png"
if not os.path.exists(output_filepath):
    os.makedirs(output_filepath)
output_file = os.path.join(output_filepath, output_filename)
plt.savefig(output_file,  dpi=400)
plt.close()

print(f"Saved @ {output_file}")

Saved @ data/withFlory/phi_g[0.4 0.3 0.2 0.1]/analysed/steps100000/distinct_mergers_4To2_Position.png


In [19]:
# Earlier versions
# merged_compartments_list = [np.array([1, 2], dtype=np.int64),
#                             np.array([1, 3], dtype=np.int64),
#                             np.array([1, 4], dtype=np.int64),
#                             np.array([2, 3], dtype=np.int64),
#                             np.array([2, 4], dtype=np.int64),
#                             np.array([3, 4], dtype=np.int64),
#                            ]

# for phi_global in phi_globals:
#     distinct_phases = []
    
#     num_Xs = len(best_F_4To3[next(iter(best_F_4To3))])  # Get length of the first key's list
    
#     for i in tqdm(range(num_Xs)):
#         # Collect F_best values for all compartment merges at this X index
#         f_at_X = []
#         for key in merged_compartments_list:
#             key_tuple = tuple(key)
#             f_at_X.append(round(best_F_4To3[key_tuple][i], 4))  
        
#         # Count distinct phases (unique F_best values)
#         distinct_phases.append(len(set(f_at_X)))



# fig, ax = plt.subplots(figsize=(8, 6))

# ax.scatter(Xs, distinct_phases, marker = "x", s = 15)

# ax.set_ylabel(r"Number of Distinct Mergers", fontsize=16, fontweight="bold")
# ax.set_xlabel(r"$\mathbf{X}$", fontsize=16, fontweight="bold")

# ax.set_xticks(Xs[::10])
# ax.set_yticks(range(0, max(distinct_phases) + 2))

# for label in ax.get_xticklabels() + ax.get_yticklabels():
#     label.set_fontweight("bold")
                        
# ax.tick_params(axis='both', which='both', direction='in', width=2, length=6, labelsize=12)
# ax.tick_params(axis='both', which='minor', direction='in', width=1.5, length=4, labelsize=10)

# ax.grid(True, which='major', linestyle='--', linewidth=1, alpha=0.6)

# plt.tight_layout()

# # output_filepath = f"data/withFlory/phi_g{phi_global}/analysed/steps{steps}/"
# # output_filename = f"distinct_mergers_4To3.png"
# # if not os.path.exists(output_filepath):
# #     os.makedirs(output_filepath)
# # output_file = os.path.join(output_filepath, output_filename)
# # plt.savefig(output_file,  dpi=400)
# # plt.close()

# # print(f"Saved @ {output_file}")

In [20]:

# merged_compartments_set1 = [np.array([1, 2], dtype=np.int64), 
#                             np.array([1, 3], dtype=np.int64),
#                             np.array([1, 4], dtype=np.int64)]

# merged_compartments_set2 = [np.array([3, 4], dtype=np.int64), 
#                             np.array([2, 4], dtype=np.int64),
#                             np.array([2, 3], dtype=np.int64)]


# merged_compartments_list = [np.array([1, 2, 3], dtype=np.int64), 
#                             np.array([1, 2, 4], dtype=np.int64),
#                             np.array([1, 3, 4], dtype=np.int64),
#                             np.array([2, 3, 4], dtype=np.int64)]

# for phi_global in phi_globals:
#     distinct_phases = []
    
#     # Get the number of X points (assuming all keys have same-length lists)
#     num_Xs = len(best_F_4To2[next(iter(best_F_4To2))])  
    
#     for i in tqdm(range(num_Xs)):
#         f_at_X = []
        
#         # Handle 4To2_combine2And2 cases (keys are tuples of tuples)
#         for merged_pair in zip(merged_compartments_set1, merged_compartments_set2):
#             key = (tuple(merged_pair[0]), tuple(merged_pair[1]))  # Ensure key is hashable
#             f_at_X.append(round(best_F_4To2[key][i], 3))
        
#         # Handle 4To2_combine3 cases (keys are plain tuples)
#         for merged in merged_compartments_list:
#             key = tuple(merged)  # Convert NumPy array to tuple
#             f_at_X.append(round(best_F_4To2[key][i], 3))
        
#         distinct_phases.append(len(set(f_at_X)))



# fig, ax = plt.subplots(figsize=(8, 6))

# ax.scatter(Xs, distinct_phases, marker = "x", s = 15)

# ax.set_ylabel(r"Number of Distinct Mergers", fontsize=16, fontweight="bold")
# ax.set_xlabel(r"$\mathbf{X}$", fontsize=16, fontweight="bold")

# ax.set_xticks(Xs[::10])
# ax.set_yticks(range(0, max(distinct_phases) + 2))


# for label in ax.get_xticklabels() + ax.get_yticklabels():
#     label.set_fontweight("bold")
                        
# ax.tick_params(axis='both', which='both', direction='in', width=2, length=6, labelsize=12)
# ax.tick_params(axis='both', which='minor', direction='in', width=1.5, length=4, labelsize=10)

# ax.grid(True, which='major', linestyle='--', linewidth=1, alpha=0.6)

# fig.tight_layout()

# # output_filepath = f"data/withFlory/phi_g{phi_global}/analysed/steps{steps}/"
# # output_filename = f"distinct_mergers_4To2.png"
# # if not os.path.exists(output_filepath):
# #     os.makedirs(output_filepath)
# # output_file = os.path.join(output_filepath, output_filename)
# # plt.savefig(output_file,  dpi=400)
# # plt.close()

# # print(f"Saved @ {output_file}")